## Setup

In [ ]:
import os
from langchain_openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pyprojroot import here
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [ ]:
load_dotenv(find_dotenv())
api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
data = PyPDFLoader(str(here("./schadenmeldung.pdf"))).load()
print(data)

## Text Representation & Chunking

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=25,
)

docs = splitter.split_documents(data)
print(docs)

In [ ]:
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

vectordb = FAISS.from_documents(
    docs,
    embedding_function,
)

vectordb.save_local(str(here("./")))

In [ ]:
qa = RetrievalQA.from_chain_type(
        llm=OpenAI(
            model_name="gpt-3.5-turbo-instruct",
            temperature=0,
            openai_api_key=api_key
        ), 
        chain_type="stuff", 
        retriever=vectordb.as_retriever()
    )

### Was muss ich mein Model fragen um die richtige Antwort zu erhalten?

In [ ]:
print(qa.invoke("Wann ist der Schaden entstanden?"))

In [ ]:
print(qa.invoke("Wann ist die Meldung eingegangen?"))

In [ ]:
print(qa.invoke("Fasse den Inhalt des Dokuments in einem Satz zusammen"))

In [ ]:
print(qa.invoke("Frage: An welchem Datum ist der Schaden entstaden? Hinweis: Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein, es kann jedoch auch ein anderes Datum sein. Gesucht wird das Datum, an dem tatsächlich etwas beschädigt wurde."))

In [ ]:
print(qa.invoke("In welche Kategorie passt die Meldung am besten? Sturm, Leitungswasser, Elementar, Einbruch, Diebstahl, Feuer, Sonstiges"))

### Je spezifischer die Frage, desto besser ist die Antwort

In [ ]:
print(qa.invoke("An welchem Datum war der Sturm, durch den der Schaden entstanden ist?"))

## Few-Shot-Approach

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
template = "context: {context} \n question: {question} \n answer: {answer}"

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["context", "question", "answer"],
    template=template,    
)

In [ ]:
examples = [
    {
        "context": """
            Betreff: Leitungswasserschaden in der Wohnung - Meldung vom 01.02.2024

            Sehr geehrte Damen und Herren,

            ich melde einen Leitungswasserschaden in meiner Wohnung in Köln-Süd (Mainzer Straße 45, 50678 Köln), der am 01.02.2024 gegen 12Uhr festgestellt wurde.

            Versicherungsnummer: GH567890123

            Der Schaden ist durch einen Defekt an einer Wasserleitung entstanden. Infolge des Lecks trat Wasser aus und beschädigte Teile der Wohnung sowie Hausrat.

            Betroffene Bereiche:

            Küche: Wasserschäden an Wänden, Boden und Fliesen
            Wohnzimmer: Wasserschäden an Parkett und Möbeln
            Schlafzimmer: Wasserschäden an Teppich und Dekoartikeln

            Betroffene Gegenstände:

                Einbauküche (Marke: X, Modell: Y, Zeitwert: 5.000€)
                Sofa (Marke: Z, Modell: W, Zeitwert: 1.000€)
                Bett (Marke: A, Modell: B, Zeitwert: 800€)

            Fotos der Schäden sind vorhanden und können auf Anfrage zur Verfügung gestellt werden.

            Ich habe bereits alle notwendigen Maßnahmen ergriffen, um weitere Schäden zu verhindern, z.B. die Wasserzufuhr abgestellt und einen Fachmann für die Trocknung der betroffenen Räume beauftragt.

            Ich bitte um eine schnelle Bearbeitung meiner Schadensmeldung und um Informationen zu weiteren Schritten.

            Rückfragen unter:

                Telefon: 0221/1234567
                E-Mail: max.mustermann@email.de

            Mit freundlichen Grüßen,    

            Max Mustermann
        """,
        "question":"Wann ist der Schaden entstanden?",
        "answer": "01.02.2024",
    },
    {
        "context": """ 
         Betreff: Meldung eines Fahrraddiebstahls

            Sehr geehrte Damen und Herren,

            hiermit melde ich den Diebstahl meines Fahrrads, welches zuletzt im Fahrradkeller meines Wohnhauses in der Düsseldorfer Straße 88, 50674 Köln, abgestellt war. Der genaue Zeitpunkt des Diebstahls ist mir leider unbekannt, da ich mein Fahrrad längere Zeit nicht benutzt habe. Zuletzt gesehen habe ich es vor etwa zwei Wochen.

            Versicherungsnummer: VF2345678910

            Es handelt sich um ein Mountainbike der Marke Trek, Modell Marlin 7, in der Farbe Rot mit schwarzem Rahmendesign. Besondere Merkmale sind die nachgerüsteten Schwalbe Reifen und ein spezieller Brooks-Sattel. Das Fahrrad hat einen Zeitwert von ca. 800€.

            Ich habe den Diebstahl bereits bei der Polizei gemeldet, und die Anzeigennummer lautet #KL5678910. Eine Kopie der Diebstahlanzeige kann ich Ihnen gerne zukommen lassen.

            Ich bitte um eine schnelle Bearbeitung meiner Schadensmeldung und um Informationen zu weiteren Schritten.

            Rückfragen stehe ich gerne unter den folgenden Kontaktdaten zur Verfügung:

                Telefon: 0211/9876543
                E-Mail: julia.schmidt@email.com

            Mit freundlichen Grüßen,

            Julia Schmidt
        """,
        "question": "Wann ist der Schaden entstanden?",
        "answer": "",
    },
]

In [ ]:
print(example_prompt.format(**examples[0]))

In [ ]:
print(example_prompt.format(**examples[1]))

In [ ]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="context: {input}",
    input_variables=["input"],
)

In [ ]:
print(few_shot_prompt_template.format(input=data[0].page_content))

In [ ]:
few_shot_chain = few_shot_prompt_template | OpenAI(
            model_name="gpt-3.5-turbo-instruct",
            temperature=0,
            openai_api_key=api_key
        )

In [ ]:
print(few_shot_chain.invoke({"input": few_shot_prompt_template.format(input=data[0].page_content)}))